# Create recipes for all dishes in the menus  

This notebook is related to the menu display 2 project, also known as exquisite menus.  


We load all the JSON menus (with all dishes) into a single large string, and then extract the dishes to generate the associated menus.

## install libs

In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


## very long flat database -- do not open!

This cell contains all the menus and all the dishes.

In [10]:
recipe = """
    [{
        "location": "Four Seasons Hotel, Paris",
        "restaurant_name": "L'Etoile d'Or",
        "chef_name": "Chef Laurent Dupont",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Crispy Parmesan waffle with tomato and basil gelée",
                "price": 15,
                "wine_pairing": "Champagne Henriot Brut Souverain"
            },
            {
                "course": "Appetizer",
                "description": "Caramelized scallops with cauliflower purée and truffle essence",
                "price": 40,
                "wine_pairing": "Sancerre Domaine Vacheron 2020"
            },
            {
                "course": "Soup",
                "description": "Velvety butternut squash bisque with crème fraîche and sage croutons",
                "price": 25,
                "wine_pairing": "Chardonnay Olivier Leflaive Bourgogne 2019"
            },
            {
                "course": "Fish",
                "description": "Pan-seared turbot with Jerusalem artichoke purée, morel mushrooms, and beurre blanc",
                "price": 55,
                "wine_pairing": "Meursault Les Narvaux Domaine Michelot 2018"
            },
            {
                "course": "Meat",
                "description": "Roasted duck breast with figs, purple potatoes, and honey-thyme reduction",
                "price": 60,
                "wine_pairing": "Pomerol Château La Pointe 2016"
            },
            {
                "course": "Cheese",
                "description": "Selection of artisanal French cheeses with quince paste and walnut bread",
                "price": 30,
                "wine_pairing": "Sauternes Château Lafaurie-Peyraguey 2017"
            },
            {
                "course": "Dessert",
                "description": "Valrhona chocolate fondant with raspberry coulis and Tahitian vanilla ice cream",
                "price": 25,
                "wine_pairing": "Banyuls Domaine La Tour Vieille 2018"
            }
        ],
        "grand_total": 250,
        "dining_room_description": "L'Etoile d'Or offers a luxurious and intimate dining experience, adorned with opulent gold accents, crystal chandeliers, and elegant silk drapery. The soft lighting and plush velvet seating create an atmosphere of romance and sophistication, while the floor-to-ceiling windows offer breathtaking views of the Eiffel Tower."
    },
{
  "location": "Four Seasons Hotel, Kyoto",
  "restaurant_name": "Sakura No Hana",
  "chef_name": "Chef Katsuro Tanaka",
  "tasting_menu": [
    {
      "course": "Sakizuke",
      "description": "Seasonal appetizer with Kyoto vegetables, tofu, and yuzu miso",
      "price": 20,
      "wine_pairing": "Dassai 23 Junmai Daiginjo Sake"
    },
    {
      "course": "Sashimi",
      "description": "Assorted fresh sashimi selection served with house-made soy sauce and wasabi",
      "price": 45,
      "wine_pairing": "Kubota Manju Junmai Daiginjo Sake"
    },
    {
      "course": "Nimono",
      "description": "Simmered seasonal fish with daikon radish, ginger, and Kyoto miso",
      "price": 40,
      "wine_pairing": "Hakutsuru Excellent Junmai Sake"
    },
    {
      "course": "Tempura",
      "description": "Crispy tempura featuring seasonal vegetables and shrimp, served with matcha salt and tentsuyu sauce",
      "price": 35,
      "wine_pairing": "Tatenokawa 18 Junmai Daiginjo Sake"
    },
    {
      "course": "Yakimono",
      "description": "Grilled Wagyu beef with shiitake mushrooms, asparagus, and a yuzu-kosho sauce",
      "price": 65,
      "wine_pairing": "Isojiman Junmai Daiginjo Sake"
    },
    {
      "course": "Gohan",
      "description": "Koshihikari rice cooked with seasonal ingredients, miso soup, and pickles",
      "price": 30,
      "wine_pairing": "Hakkaisan Junmai Ginjo Sake"
    },
    {
      "course": "Dessert",
      "description": "Matcha mousse with azuki bean paste and yuzu sorbet",
      "price": 20,
      "wine_pairing": "Umetsu Yuzushu"
    }
  ],
  "grand_total": 255,
  "dining_room_description": "Sakura No Hana provides an exquisite and serene dining experience, featuring traditional Japanese architecture and design elements. Guests are surrounded by natural wood and stone, with delicate cherry blossom motifs adorning the walls. The floor-to-ceiling windows overlook a beautiful Japanese garden, creating an atmosphere of tranquility and harmony."
},

{
  "location": "Four Seasons Resort, Bali at Jimbaran Bay",
  "restaurant_name": "Taman Sari",
  "chef_name": "Chef Made Agus",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Balinese spiced tuna tartare on crispy cassava chip",
      "price": 15,
      "wine_pairing": "Cloudy Bay Sauvignon Blanc 2020"
    },
    {
      "course": "Appetizer",
      "description": "Gado-gado salad with peanut sauce and fresh local vegetables",
      "price": 25,
      "wine_pairing": "Domaine Laroche Chablis Saint Martin 2019"
    },
    {
      "course": "Soup",
      "description": "Soto Ayam, a traditional Indonesian chicken soup with lemongrass, turmeric, and glass noodles",
      "price": 20,
      "wine_pairing": "Dr. Loosen Blue Slate Riesling Kabinett 2019"
    },
    {
      "course": "Fish",
      "description": "Ikan Bakar, grilled Jimbaran Bay fish with sambal matah and steamed rice",
      "price": 45,
      "wine_pairing": "Pascal Jolivet Pouilly-Fumé 2020"
    },
    {
      "course": "Meat",
      "description": "Babi Guling, Balinese roast suckling pig with urap vegetables and lawar salad",
      "price": 55,
      "wine_pairing": "Penfolds Bin 28 Kalimna Shiraz 2018"
    },
    {
      "course": "Dessert",
      "description": "Dadar Gulung, coconut crepes with palm sugar syrup and vanilla ice cream",
      "price": 20,
      "wine_pairing": "Château Rieussec Sauternes 2017"
    }
  ],
  "grand_total": 180,
  "dining_room_description": "Taman Sari, meaning 'Water Garden' in Indonesian, offers a breathtaking open-air dining experience. The restaurant is adorned with traditional Balinese carvings, lush tropical greenery, and water features. Guests can enjoy the stunning ocean views while indulging in the flavors of Bali, as the gentle sea breeze adds to the enchanting atmosphere."
},

{
  "location": "Four Seasons Resort, Bora Bora",
  "restaurant_name": "Le Lagon",
  "chef_name": "Chef Matthieu Guérin",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Tahitian-style poisson cru with coconut milk and lime",
      "price": 20,
      "wine_pairing": "Château d'Esclans Whispering Angel Rosé 2020"
    },
    {
      "course": "Appetizer",
      "description": "Grilled octopus with mango salsa and vanilla-infused olive oil",
      "price": 35,
      "wine_pairing": "Dog Point Vineyard Sauvignon Blanc 2020"
    },
    {
      "course": "Soup",
      "description": "Chilled cucumber and pineapple soup with mint and rum-infused yogurt",
      "price": 20,
      "wine_pairing": "Robert Mondavi Fumé Blanc 2019"
    },
    {
      "course": "Fish",
      "description": "Pan-seared mahi-mahi with sweet potato purée, sautéed bok choy, and passionfruit beurre blanc",
      "price": 50,
      "wine_pairing": "Domaine Louis Moreau Chablis 2019"
    },
    {
      "course": "Meat",
      "description": "Grilled New Zealand lamb chops with taro root gratin, broccolini, and rosemary jus",
      "price": 60,
      "wine_pairing": "Craggy Range Te Kahu Gimblett Gravels Vineyard 2018"
    },
    {
      "course": "Dessert",
      "description": "Vanilla bean panna cotta with poached papaya and coconut tuile",
      "price": 20,
      "wine_pairing": "Royal Tokaji Late Harvest 2017"
    }
  ],
  "grand_total": 205,
  "dining_room_description": "Le Lagon offers an unparalleled tropical dining experience, with a stunning open-air setting overlooking the turquoise waters of Bora Bora's lagoon. Surrounded by lush vegetation, the restaurant features Polynesian-inspired design elements, such as thatched-roof pavilions, hand-carved sculptures, and warm lighting. Guests can savor the exquisite cuisine while enjoying the gentle ocean breeze and breathtaking views of Mount Otemanu."
},

{
  "location": "Four Seasons Resort, Sacred Valley, Peru",
  "restaurant_name": "Inti Raymi",
  "chef_name": "Chef Alejandro Sánchez",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Purple corn crisp with avocado mousse and trout ceviche",
      "price": 15,
      "wine_pairing": "Veramonte Sauvignon Blanc 2020"
    },
    {
      "course": "Appetizer",
      "description": "Peruvian causa with layers of potato, aji amarillo, and crab salad",
      "price": 25,
      "wine_pairing": "Terrazas de los Andes Reserva Torrontés 2019"
    },
    {
      "course": "Soup",
      "description": "Quinoa and Andean cheese chowder with roasted corn and huacatay",
      "price": 20,
      "wine_pairing": "Tabernero Gran Tinto Petit Verdot 2018"
    },
    {
      "course": "Fish",
      "description": "Amazonian paiche with aji panca glaze, yucca purée, and sautéed kale",
      "price": 45,
      "wine_pairing": "Tikal Natural Mendoza Blend 2017"
    },
    {
      "course": "Meat",
      "description": "Alpaca tenderloin with Peruvian corn succotash and rocoto pepper sauce",
      "price": 55,
      "wine_pairing": "Enrique Foster Reserva Malbec 2018"
    },
    {
      "course": "Dessert",
      "description": "Lucuma mousse with maracuya coulis and chocolate crumble",
      "price": 20,
      "wine_pairing": "Susana Balbo Late Harvest Malbec 2017"
    }
  ],
  "grand_total": 180,
  "dining_room_description": "Inti Raymi, named after the Inca festival celebrating the sun god, offers a luxurious dining experience that embraces the rich cultural heritage of the Sacred Valley. The dining room features rustic stone walls, high wooden ceilings, and warm lighting, creating an atmosphere of sophistication and tranquility. Floor-to-ceiling windows showcase breathtaking views of the Andes Mountains, while local textiles and artwork pay homage to the region's indigenous roots."
},

{
  "location": "Four Seasons Safari Lodge, Serengeti, Tanzania",
  "restaurant_name": "Majani",
  "chef_name": "Chef Amani Sudi",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Biltong-spiced beef carpaccio with microgreens and lemon vinaigrette",
      "price": 20,
      "wine_pairing": "Hamilton Russell Vineyards Chardonnay 2020"
    },
    {
      "course": "Appetizer",
      "description": "Grilled tiger prawns with mango and avocado salad, and a chili-lime dressing",
      "price": 30,
      "wine_pairing": "Mulderbosch Chenin Blanc 2019"
    },
    {
      "course": "Soup",
      "description": "Roasted butternut squash soup with coconut cream, pumpkin seeds, and coriander",
      "price": 20,
      "wine_pairing": "Warwick The First Lady Unoaked Chardonnay 2020"
    },
    {
      "course": "Fish",
      "description": "Pan-seared Nile perch with okra, tomatoes, and an aromatic tamarind sauce",
      "price": 45,
      "wine_pairing": "Cederberg Sauvignon Blanc 2020"
    },
    {
      "course": "Meat",
      "description": "Safari-spiced ostrich fillet with sweet potato mash, sautéed spinach, and red wine reduction",
      "price": 55,
      "wine_pairing": "Rust en Vrede Estate Cabernet Sauvignon 2018"
    },
    {
      "course": "Dessert",
      "description": "Amarula-infused chocolate fondant with raspberry coulis and vanilla ice cream",
      "price": 25,
      "wine_pairing": "Delaire Graff Botmaskop 2017"
    }
  ],
  "grand_total": 195,
  "dining_room_description": "Majani, meaning 'green' in Swahili, offers a luxurious and unique dining experience amidst the breathtaking landscapes of the Serengeti. The dining room features elegant wood furnishings, warm lighting, and African-inspired décor. Large open-air terraces allow guests to savor the flavors of the region while enjoying the magnificent views of the surrounding savannah and the majestic wildlife that inhabits it."
},

{
  "location": "Four Seasons Hotel, Venice",
  "restaurant_name": "La Serenissima",
  "chef_name": "Chef Giuseppe Di Martino",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Crispy polenta with black truffle and Parmigiano-Reggiano foam",
      "price": 25,
      "wine_pairing": "Ca' del Bosco Franciacorta Cuvée Prestige Brut NV"
    },
    {
      "course": "Appetizer",
      "description": "Seared scallops with artichoke purée, crispy pancetta, and lemon-infused olive oil",
      "price": 45,
      "wine_pairing": "Livio Felluga Terre Alte 2018"
    },
    {
      "course": "Pasta",
      "description": "Homemade squid ink tagliolini with lobster, cherry tomatoes, and basil",
      "price": 55,
      "wine_pairing": "Pieropan Soave Classico La Rocca 2017"
    },
    {
      "course": "Fish",
      "description": "Pan-seared turbot with fennel confit, olives, and a saffron-orange emulsion",
      "price": 65,
      "wine_pairing": "Planeta Chardonnay 2019"
    },
    {
      "course": "Meat",
      "description": "Veal tenderloin with porcini mushrooms, truffled potato purée, and Barolo reduction",
      "price": 75,
      "wine_pairing": "Brunello di Montalcino Poggio Antico 2015"
    },
    {
      "course": "Dessert",
      "description": "Tiramisù semifreddo with espresso sauce and cocoa nibs",
      "price": 30,
      "wine_pairing": "Moscato d'Asti La Spinetta Bricco Quaglia 2020"
    }
  ],
  "grand_total": 295,
  "dining_room_description": "La Serenissima, named after the historic Republic of Venice, provides an unparalleled dining experience within a lavish and elegant setting. The dining room boasts magnificent Venetian chandeliers, intricately frescoed ceilings, and luxurious silk wall coverings. Large windows overlooking the Grand Canal offer guests an enchanting view of Venice's timeless beauty as they indulge in the exquisite cuisine and fine Italian wines."
},

{
  "location": "Four Seasons Hotel George V, Paris",
  "restaurant_name": "Le Paradis des Douceurs",
  "chef_name": "Chef Juliette Dubois",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Mini raspberry macarons with rose petal cream",
      "price": 15,
      "wine_pairing": "Château d'Yquem Sauternes 2017"
    },
    {
      "course": "Appetizer",
      "description": "Exotic fruit carpaccio with lime zest and mint-infused syrup",
      "price": 25,
      "wine_pairing": "Domaine Huet Vouvray Moelleux Le Mont 2019"
    },
    {
      "course": "Soufflé",
      "description": "Grand Marnier soufflé with candied orange peel",
      "price": 35,
      "wine_pairing": "Domaine Disznókő Tokaji Aszú 5 Puttonyos 2015"
    },
    {
      "course": "Tarte",
      "description": "Tarte Tatin with caramelized apples, puff pastry, and vanilla crème fraîche",
      "price": 30,
      "wine_pairing": "Château Suduiraut Sauternes 2016"
    },
    {
      "course": "Mousse",
      "description": "Dark chocolate mousse with hazelnut praline and gold leaf",
      "price": 30,
      "wine_pairing": "Domaine de Rancy Rivesaltes Ambré Hors d'Age 2000"
    },
    {
      "course": "Petit Fours",
      "description": "Assorted French petit fours: éclairs, madeleines, and financiers",
      "price": 25,
      "wine_pairing": "Château La Rame Sainte-Croix-du-Mont 2017"
    }
  ],
  "grand_total": 160,
  "dining_room_description": "Le Paradis des Douceurs, situated within the opulent Four Seasons Hotel George V, transports guests to a world of indulgence and delight. The dining room features elegant crystal chandeliers, luxurious fabrics, and intricate moldings that reflect the grandeur of Parisian architecture. Floor-to-ceiling windows provide a stunning view of the hotel's beautiful gardens, creating an enchanting atmosphere in which to savor the exquisite dessert creations and fine wine pairings."
},

{
  "location": "Four Seasons Resort, French Alps",
  "restaurant_name": "L'Étoile des Neiges",
  "chef_name": "Chef Élodie Moreau",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Beetroot-cured salmon with horseradish cream and microgreens",
      "price": 20,
      "wine_pairing": "Domaine Zind-Humbrecht Riesling 2019"
    },
    {
      "course": "Appetizer",
      "description": "Seared foie gras with spiced pear chutney and toasted brioche",
      "price": 40,
      "wine_pairing": "Château Rieussec Sauternes 2017"
    },
    {
      "course": "Soup",
      "description": "Creamy wild mushroom soup with crispy bacon and chives",
      "price": 25,
      "wine_pairing": "Joseph Drouhin Côte de Beaune 2018"
    },
    {
      "course": "Pasta",
      "description": "Savoyard-style gnocchi with Reblochon cheese, lardons, and caramelized onions",
      "price": 35,
      "wine_pairing": "Louis Jadot Moulin-à-Vent 2018"
    },
    {
      "course": "Meat",
      "description": "Roasted venison loin with parsnip purée, Brussels sprouts, and juniper berry sauce",
      "price": 55,
      "wine_pairing": "Château de Beaucastel Châteauneuf-du-Pape 2019"
    },
    {
      "course": "Dessert",
      "description": "Warm chocolate fondant with raspberry coulis and pistachio ice cream",
      "price": 25,
      "wine_pairing": "Banyuls Domaine La Tour Vieille Rimage 2017"
    }
  ],
  "grand_total": 200,
  "dining_room_description": "L'Étoile des Neiges offers a luxurious dining experience amidst the breathtaking surroundings of the French Alps. The dining room features warm wood accents, an elegant stone fireplace, and plush seating, creating a cozy ambiance. Large windows provide panoramic views of the snow-capped mountains, allowing guests to savor the exquisite cuisine and carefully selected wine pairings while immersed in the enchanting alpine landscape."
},

{
  "location": "Four Seasons Hotel, Helsinki, Finland",
  "restaurant_name": "Nordic Essence",
  "chef_name": "Chef Akseli Korhonen",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Crispy reindeer moss with mushroom powder and fermented lingonberries",
      "price": 50,
      "wine_pairing": "Domäne Wachau Riesling Terrassen Federspiel 2019"
    },
    {
      "course": "Appetizer",
      "description": "Smoked eel with pickled cucumber, horseradish cream, and dill oil",
      "price": 80,
      "wine_pairing": "Pascal Jolivet Sancerre 2020"
    },
    {
      "course": "Fish",
      "description": "Roasted Arctic char with beetroot purée, preserved lemon, and sea buckthorn",
      "price": 100,
      "wine_pairing": "Pierre-Yves Colin-Morey Chassagne-Montrachet 2019"
    },
    {
      "course": "Bird",
      "description": "Wild duck breast with caramelized celeriac, blackcurrants, and a juniper berry jus",
      "price": 120,
      "wine_pairing": "Georges Vernay Côte-Rôtie Blonde du Seigneur 2018"
    },
    {
      "course": "Meat",
      "description": "Elk tartare with smoked egg yolk, caper berries, and foraged herbs",
      "price": 110,
      "wine_pairing": "Tenuta San Guido Sassicaia 2017"
    },
    {
      "course": "Cheese",
      "description": "Selection of Nordic cheeses with Finnish cloudberry jam and crisp rye bread",
      "price": 60,
      "wine_pairing": "Château Climens Barsac 2016"
    },
    {
      "course": "Dessert",
      "description": "Birch sap sorbet with wild blueberries and spruce needle meringue",
      "price": 80,
      "wine_pairing": "Domaine Huet Vouvray Moelleux Le Mont 2019"
    }
  ],
  "grand_total": 600,
  "dining_room_description": "Nordic Essence, inspired by the innovative culinary techniques and flavors of Noma and other renowned Nordic restaurants, offers a truly unique and immersive dining experience in the heart of Helsinki. The dining room showcases elegant modern design with natural elements, reflecting the beauty of Finland's pristine landscapes. Large windows overlook the city's stunning architecture, while warm lighting and delicate textiles create an intimate ambiance in which to indulge in the extraordinary tasting menu and expertly curated wine pairings."
},

{
  "location": "Four Seasons Hotel, Berlin, Germany",
  "restaurant_name": "Deutscher Geschmack",
  "chef_name": "Chef Lukas Müller",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Miniature potato pancake with smoked salmon and horseradish cream",
      "price": 20,
      "wine_pairing": "Dr. Loosen Riesling Sekt Brut 2018"
    },
    {
      "course": "Appetizer",
      "description": "Pan-seared duck liver with apple compote, brioche, and aged balsamic",
      "price": 45,
      "wine_pairing": "Donnhoff Oberhäuser Leistenberg Riesling Kabinett 2019"
    },
    {
      "course": "Soup",
      "description": "Creamy white asparagus soup with chervil and toasted almonds",
      "price": 30,
      "wine_pairing": "Markus Molitor Zeltinger Sonnenuhr Riesling Spätlese 2018"
    },
    {
      "course": "Fish",
      "description": "Braised monkfish with cabbage, bacon, and Riesling cream sauce",
      "price": 60,
      "wine_pairing": "Franz Hirtzberger Rotes Tor Grüner Veltliner Federspiel 2019"
    },
    {
      "course": "Meat",
      "description": "Venison medallions with red cabbage, spätzle, and lingonberry sauce",
      "price": 70,
      "wine_pairing": "August Kesseler Pinot Noir 'R' 2017"
    },
    {
      "course": "Dessert",
      "description": "Black Forest cake with cherry compote and kirsch-infused whipped cream",
      "price": 35,
      "wine_pairing": "Müller-Catoir Haardter Bürgergarten Rieslaner Auslese 2017"
    }
  ],
  "grand_total": 260,
  "dining_room_description": "Deutscher Geschmack celebrates the rich culinary heritage of Germany with a modern and refined twist. The dining room is a blend of contemporary elegance and traditional German design, featuring dark wood accents, plush seating, and warm lighting. Large windows provide a view of the bustling city of Berlin, while the intimate atmosphere allows guests to savor the superbly crafted tasting menu and carefully chosen wine pairings."
},

{
  "location": "Four Seasons Hotel, Mars Base One",
  "restaurant_name": "The Red Planet Bistro",
  "chef_name": "Chef Elon Ray",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Martian soil-cultivated microgreen salad with synthesized honey-mustard vinaigrette",
      "price": 200,
      "wine_pairing": "Terran Vineyards Earth-to-Mars Cabernet Sauvignon 2048"
    },
    {
      "course": "Appetizer",
      "description": "Lab-grown shrimp cocktail with spicy tomato gel and hydroponic dill",
      "price": 300,
      "wine_pairing": "Lunar Valley Chardonnay 2049"
    },
    {
      "course": "Soup",
      "description": "Cream of Martian potato soup with cultured sour cream and bio-printed chives",
      "price": 250,
      "wine_pairing": "Phobos Estate Mars-grown Riesling 2047"
    },
    {
      "course": "Main Course",
      "description": "Cellular agriculture steak with Mars-grown mushroom sauce, hydroponic vegetables, and red dust-infused demi-glace",
      "price": 500,
      "wine_pairing": "Deimos Heights Pinot Noir 2050"
    },
    {
      "course": "Dessert",
      "description": "Galactic chocolate mousse with edible holographic fruit garnish",
      "price": 400,
      "wine_pairing": "Andromeda Winery Late Harvest Nebula Nectar 2045"
    }
  ],
  "grand_total": 1650,
  "dining_room_description": "The Red Planet Bistro at Four Seasons Mars Base One presents an out-of-this-world dining experience like no other. The dining room features panoramic windows overlooking the Martian landscape, allowing guests to marvel at the red planet's unique beauty while savoring the innovative cuisine. The design seamlessly blends cutting-edge technology with the comfort and luxury expected from the Four Seasons brand. Enjoy the specially crafted tasting menu, showcasing the best of both Earth-imported and Mars-grown ingredients, paired with an exquisite selection of interstellar wines."
},

{
  "location": "Four Seasons Hotel, New York City",
  "restaurant_name": "Verdant Elegance",
  "chef_name": "Chef Isabella Green",
  "tasting_menu": [
    {
      "course": "Amuse-Bouche",
      "description": "Heirloom tomato tartare with basil microgreens and pine nut 'Parmesan'",
      "price": 150,
      "wine_pairing": "Domaine Leflaive Puligny-Montrachet 2018"
    },
    {
      "course": "Appetizer",
      "description": "Wild mushroom carpaccio with black truffle vinaigrette and vegan 'foie gras'",
      "price": 250,
      "wine_pairing": "Pahlmeyer Napa Valley Chardonnay 2017"
    },
    {
      "course": "Soup",
      "description": "Chilled sweet corn velouté with avocado mousse and chipotle-infused oil",
      "price": 200,
      "wine_pairing": "Quintessa Illumination Sauvignon Blanc 2019"
    },
    {
      "course": "Pasta",
      "description": "House-made artichoke and cashew ricotta tortellini with sun-dried tomato sauce and crispy sage",
      "price": 300,
      "wine_pairing": "Gaja Barbaresco 2016"
    },
    {
      "course": "Main Course",
      "description": "Pan-seared black garlic and porcini-crusted king oyster mushroom 'scallops' with saffron-infused cauliflower purée and microgreens",
      "price": 400,
      "wine_pairing": "Domaine de la Romanée-Conti Échézeaux 2015"
    },
    {
      "course": "Dessert",
      "description": "Triple-layered dark chocolate and hazelnut mousse cake with edible gold leaf and raspberry coulis",
      "price": 200,
      "wine_pairing": "Krug Clos d'Ambonnay 2000"
    },
    {
      "course": "Petit Fours",
      "description": "Assorted vegan petit fours: passion fruit truffles, matcha macarons, and raspberry-filled chocolates",
      "price": 200,
      "wine_pairing": "Louis Roederer Cristal 2012"
    }
  ],
  "grand_total": 1200,
  "dining_room_description": "Verdant Elegance is a three-Michelin-star vegan restaurant located within the prestigious Four Seasons Hotel in New York City. The dining room features floor-to-ceiling windows with stunning views of the city skyline, creating a sophisticated atmosphere. Luxurious furnishings, contemporary design, and an unparalleled commitment to sustainable, plant-based dining make Verdant Elegance a truly unforgettable experience. Indulge in the meticulously crafted tasting menu, showcasing innovative vegan cuisine, and savor the expertly curated wine pairings from a world-class sommelier."
},

{
  "location": "Four Seasons Resort Maldives at Kuda Huraa",
  "restaurant_name": "Fisherman's Grill",
  "chef_name": "Chef Ahmed Naseer",
  "tasting_menu": [
    {
      "course": "Appetizer",
      "description": "Grilled octopus with cherry tomatoes, kalamata olives, and lemon-garlic dressing",
      "price": 60,
      "wine_pairing": "Cloudy Bay Sauvignon Blanc 2020"
    },
    {
      "course": "Fish",
      "description": "Charcoal-grilled red snapper with tamarind glaze, mango salsa, and coconut rice",
      "price": 90,
      "wine_pairing": "Clos des Papes Châteauneuf-du-Pape Blanc 2018"
    },
    {
      "course": "Seafood",
      "description": "Mixed seafood grill with jumbo prawns, calamari, and scallops, served with a spicy tomato sauce",
      "price": 110,
      "wine_pairing": "Paul Jaboulet Aîné Hermitage La Chapelle 2015"
    },
    {
      "course": "Meat",
      "description": "Grilled lamb chops with rosemary marinade, ratatouille, and mint yogurt",
      "price": 120,
      "wine_pairing": "Château Mouton Rothschild 2012"
    },
    {
      "course": "Dessert",
      "description": "Coconut panna cotta with passion fruit coulis and toasted coconut flakes",
      "price": 30,
      "wine_pairing": "Domaine des Baumard Quarts de Chaume 2017"
    },
    {
      "course": "Dessert",
      "description": "Grilled pineapple with rum caramel sauce and vanilla ice cream",
      "price": 30,
      "wine_pairing": "Château Suduiraut Sauternes 2015"
    },
    {
      "course": "Dessert",
      "description": "Warm chocolate lava cake with coconut ice cream and macadamia nuts",
      "price": 30,
      "wine_pairing": "Ramos Pinto Quinta do Bom Retiro 20 Year Tawny Port"
    }
  ],
  "grand_total": 470,
  "dining_room_description": "Fisherman's Grill at Four Seasons Resort Maldives at Kuda Huraa is a seaside restaurant specializing in freshly grilled fish and seafood. The dining room is open-air with panoramic ocean views, creating a relaxed and inviting atmosphere. Enjoy a delectable tasting menu showcasing the best local catches and expertly paired wines, surrounded by the natural beauty of the Maldives. With its attentive service, superb cuisine, and breathtaking setting, Fisherman's Grill is a dining experience not to be missed."
},

{
"location": "Four Seasons Hotel Lion Palace, St. Petersburg, Russia",
"restaurant_name": "Imperial Caviar",
"chef_name": "Chef Dimitri Petrov",
"tasting_menu": [
{
"course": "Amuse-Bouche",
"description": "Blini with smoked salmon and Ossetra caviar",
"price": 20,
"wine_pairing": "Champagne Taittinger Brut Réserve"
},
{
"course": "Appetizer",
"description": "Beetroot and vodka-cured sturgeon with horseradish cream",
"price": 35,
"wine_pairing": "Chablis Domaine Laroche Saint Martin 2019"
},
{
"course": "Soup",
"description": "Traditional Russian borscht with sour cream and pumpernickel bread",
"price": 30,
"wine_pairing": "Pinot Noir Domaine Gille Bourgogne 2018"
},
{
"course": "Fish",
"description": "Pan-fried halibut with celeriac puree, wilted spinach, and dill beurre blanc",
"price": 55,
"wine_pairing": "Sauvignon Blanc Cloudy Bay Marlborough 2020"
},
{
"course": "Meat",
"description": "Venison medallions with wild mushrooms, truffled potato gratin, and juniper berry sauce",
"price": 60,
"wine_pairing": "Barolo G.D. Vajra Bricco delle Viole 2016"
},
{
"course": "Cheese",
"description": "Selection of Russian artisanal cheeses with lingonberry jam and rye crackers",
"price": 30,
"wine_pairing": "Cahors Château du Cèdre 2017"
},
{
"course": "Dessert",
"description": "Honey cake with sour cream frosting and raspberry coulis",
"price": 25,
"wine_pairing": "Tokaji Aszú 5 Puttonyos Disznókő 2013"
}
],
"grand_total": 255,
"dining_room_description": "Imperial Caviar showcases the opulence of Russia's culinary heritage with lavish interiors featuring rich red velvet, sparkling crystal chandeliers, and floor-to-ceiling windows overlooking the Winter Palace. The restaurant pays homage to the grandeur of the tsarist era with its impeccable service and indulgent menu offerings."
},

  {
        "location": "Four Seasons Hotel Utopia, Dubai, United Arab Emirates",
        "restaurant_name": "Alcazar",
        "chef_name": "Chef Farid Al-Rashid",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Dates stuffed with goat cheese and crushed pistachios",
                "price": 20,
                "wine_pairing": "Champagne Ruinart Blanc de Blancs"
            },
            {
                "course": "Appetizer",
                "description": "Pan-seared scallops with saffron and orange-infused cauliflower purée",
                "price": 40,
                "wine_pairing": "Vermentino di Sardegna Cantina del Vermentino Funtanaliras 2020"
            },
            {
                "course": "Soup",
                "description": "Creamy pumpkin and coconut soup with sumac-spiced croutons",
                "price": 25,
                "wine_pairing": "Viognier Yves Cuilleron Les Vignes d'à Côté 2019"
            },
            {
                "course": "Fish",
                "description": "Seared sea bass with chickpea and spinach stew, pomegranate reduction",
                "price": 55,
                "wine_pairing": "Chardonnay Cakebread Cellars Napa Valley 2019"
            },
            {
                "course": "Meat",
                "description": "Slow-cooked lamb shank with apricot and almond couscous, harissa jus",
                "price": 60,
                "wine_pairing": "Syrah E. Guigal Côtes du Rhône 2017"
            },
            {
                "course": "Cheese",
                "description": "Selection of Middle Eastern cheeses with fig jam and za'atar flatbread",
                "price": 30,
                "wine_pairing": "Rosé Domaine Tempier Bandol 2020"
            },
            {
                "course": "Dessert",
                "description": "Rosewater and cardamom panna cotta with honeyed pistachio crumble",
                "price": 25,
                "wine_pairing": "Moscato d'Asti Vietti Cascinetta 2021"
            }
        ],
        "grand_total": 255,
        "dining_room_description": "Alcazar is a luxurious dining oasis that combines modern elegance with the rich culinary traditions of the Middle East. The opulent interiors are adorned with gold accents, intricate mosaics, and plush seating. Floor-to-ceiling windows offer sweeping views of Dubai's skyline and the glittering Arabian Gulf."
    },
    {
        "location": "Four Seasons Hotel Aurora, Reykjavik, Iceland",
        "restaurant_name": "Nordic Haven",
        "chef_name": "Chef Sigurjón Ólafsson",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Crispy Icelandic seaweed with foraged herbs and pickled berries",
                "price": 20,
                "wine_pairing": "Champagne Billecart-Salmon Brut Réserve"
            },
            {
                "course": "Appetizer",
                "description": "Cured Arctic char with dill cream, cucumber, and rye crisps",
                "price": 40,
                "wine_pairing": "Riesling Trimbach Cuvée Frédéric Emile 2014"
            },
            {
                "course": "Soup",
                "description": "Wild mushroom and barley soup with smoked skyr and toasted hazelnuts",
                "price": 25,
                "wine_pairing": "Grüner Veltliner Hirsch Kammern Kammerner Heiligenstein 2018"
            },
            {
                "course": "Fish",
                "description": "Pan-roasted cod with foraged seaweed, leeks, and mussel emulsion",
                "price": 55,
                "wine_pairing": "Chenin Blanc François Chidaine Montlouis-sur-Loire Les Choisilles 2019"
            },
            {
                "course": "Meat",
                "description": "Roasted Icelandic lamb loin with fermented root vegetables and juniper jus",
                "price": 60,
                "wine_pairing": "Pinot Noir Domaine Drouhin Oregon Dundee Hills 2017"
            },
            {
                "course": "Cheese",
                "description": "Selection of Icelandic artisanal cheeses with cloudberry compote and seed crackers",
                "price": 30,
                "wine_pairing": "Gewürztraminer Zind-Humbrecht Alsace 2019"
            },
            {
                "course": "Dessert",
                "description": "Birch syrup and caramelized skyr mousse with blueberry sorbet",
                "price": 25,
                "wine_pairing": "Ice Cider Domaine Neige Première 2017"
            }
        ],
        "grand_total": 255,
        "dining_room_description": "Nordic Haven is an enchanting dining experience that showcases the natural beauty and unique flavors of Iceland. The minimalist, yet elegant, dining room features floor-to-ceiling windows that provide panoramic views of the aurora borealis and the surrounding rugged landscape. The innovative menu highlights the best of Nordic cuisine, with a focus on local and seasonal ingredients."
    },

      {
        "location": "Four Seasons Hotel Siena, Tuscany, Italy",
        "restaurant_name": "La Dolce Vita",
        "chef_name": "Chef Alessio Bianchi",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Parmesan and balsamic pearls on a crispy rosemary cracker",
                "price": 20,
                "wine_pairing": "Franciacorta Ca' del Bosco Cuvée Prestige"
            },
            {
                "course": "Appetizer",
                "description": "Deconstructed caprese salad with buffalo mozzarella foam and tomato essence",
                "price": 40,
                "wine_pairing": "Verdicchio dei Castelli di Jesi Classico Superiore Bucci 2019"
            },
            {
                "course": "Soup",
                "description": "Smooth artichoke soup with crispy pancetta and black truffle shavings",
                "price": 25,
                "wine_pairing": "Pinot Grigio Livio Felluga Colli Orientali del Friuli 2020"
            },
            {
                "course": "Pasta",
                "description": "Agnolotti filled with slow-cooked ossobuco, saffron-infused béchamel, and gremolata",
                "price": 50,
                "wine_pairing": "Barbaresco Produttori del Barbaresco Montefico 2016"
            },
            {
                "course": "Fish",
                "description": "Pan-seared sea bass with fennel purée, orange segments, and caper-olive salsa",
                "price": 55,
                "wine_pairing": "Falanghina Feudi di San Gregorio 2020"
            },
            {
                "course": "Meat",
                "description": "Guinea fowl saltimbocca with sage, prosciutto, and Marsala wine reduction",
                "price": 60,
                "wine_pairing": "Chianti Classico Fontodi 2018"
            },
            {
                "course": "Cheese",
                "description": "Assortment of Italian artisanal cheeses with fig marmalade and rosemary focaccia",
                "price": 30,
                "wine_pairing": "Amarone della Valpolicella Classico Allegrini 2016"
            },
            {
                "course": "Dessert",
                "description": "Tiramisù semifreddo with coffee-soaked ladyfingers and cocoa nib tuile",
                "price": 25,
                "wine_pairing": "Vin Santo del Chianti Classico Felsina 2009"
            }
        ],
        "grand_total": 305,
        "dining_room_description": "La Dolce Vita offers a harmonious blend of traditional and contemporary Italian cuisine in a breathtaking setting. The dining room is adorned with elegant furnishings, warm lighting, and a frescoed ceiling that evokes the beauty of the Tuscan countryside. Expansive windows overlook the rolling hills and vineyards, creating an atmosphere of serene indulgence."
    },

    {
        "location": "Four Seasons Resort Azure Isle, Pacific Ocean",
        "restaurant_name": "Island Elysium",
        "chef_name": "Chef Luna Martin",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Smoked island fish rillettes with pickled sea beans and crispy taro chips",
                "price": 35,
                "wine_pairing": "Champagne Krug Grande Cuvée"
            },
            {
                "course": "Appetizer",
                "description": "Seared island octopus with preserved lemon, wild greens, and coconut aïoli",
                "price": 65,
                "wine_pairing": "Chablis 1er Cru Domaine William Fèvre Montée de Tonnerre 2018"
            },
            {
                "course": "Soup",
                "description": "Roasted root vegetable bisque with smoked paprika and crispy island yam",
                "price": 45,
                "wine_pairing": "Sauvignon Blanc Cloudy Bay Te Koko 2017"
            },
            {
                "course": "Fish",
                "description": "Grilled island catch with banana leaf, ginger-turmeric relish, and preserved mango",
                "price": 85,
                "wine_pairing": "Sancerre Domaine Vacheron Belle Dame 2019"
            },
            {
                "course": "Meat",
                "description": "Island-raised pork tenderloin with tamarind glaze, breadfruit purée, and pickled pineapple",
                "price": 95,
                "wine_pairing": "Pinot Noir Domaine de la Romanée-Conti Echezeaux Grand Cru 2016"
            },
            {
                "course": "Vegetarian",
                "description": "Island-grown jackfruit and black bean stew with coconut rice and plantain chips",
                "price": 65,
                "wine_pairing": "Chenin Blanc François Chidaine Montlouis-sur-Loire Les Bournais 2019"
            },
            {
                "course": "Cheese",
                "description": "Selection of island-made cheeses with spiced fruit chutney and cassava crackers",
                "price": 55,
                "wine_pairing": "Madeira Blandy's 10-Year-Old Sercial"
            },
            {
                "course": "Dessert",
                "description": "Island coconut and passion fruit panna cotta with hibiscus syrup",
                "price": 45,
                "wine_pairing": "Moscato d'Asti G.D. Vajra 2020"
            }
        ],
        "grand_total": 490,
        "dining_room_description": "Island Elysium is an exclusive dining experience that offers unparalleled tranquility and privacy on the secluded Azure Isle. Accessible only by private jet, this prestigious three-Michelin-starred restaurant highlights the island's natural bounty, with a focus on locally grown, preserved, and smoked ingredients. The intimate dining room, surrounded by lush tropical foliage and breathtaking ocean views, provides an unforgettable culinary journey into the heart of paradise."
    },

    {
        "location": "Four Seasons Hotel Tokyo, Japan",
        "restaurant_name": "Le Jardin Zen",
        "chef_name": "Chef Yuki Morimoto",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Tuna tartare with yuzu, avocado, and sesame tuile",
                "price": 20,
                "wine_pairing": "Champagne Taittinger Brut La Française"
            },
            {
                "course": "Cold Appetizer",
                "description": "Sashimi-style scallops with beurre blanc and wakame salad",
                "price": 40,
                "wine_pairing": "Sauvignon Blanc Pascal Jolivet Sancerre 2020"
            },
            {
                "course": "Hot Appetizer",
                "description": "Grilled foie gras with dashi glaze and pickled daikon radish",
                "price": 50,
                "wine_pairing": "Chardonnay Joseph Drouhin Puligny-Montrachet 2018"
            },
            {
                "course": "Soup",
                "description": "Miso-infused French onion soup with comté cheese crouton",
                "price": 30,
                "wine_pairing": "Pinot Gris Domaine Zind-Humbrecht 2019"
            },
            {
                "course": "Salad",
                "description": "Salad Niçoise with seared tuna, quail egg, and yuzu-wasabi vinaigrette",
                "price": 35,
                "wine_pairing": "Rosé Château d'Esclans Whispering Angel 2020"
            },
            {
                "course": "Fish",
                "description": "Baked cod with miso and mushroom duxelles, sautéed spinach",
                "price": 55,
                "wine_pairing": "Riesling Domaine Weinbach Cuvée Ste. Catherine 2019"
            },
            {
                "course": "Meat",
                "description": "Beef bourguignon with shiitake mushrooms, pearl onions, and soba noodles",
                "price": 60,
                "wine_pairing": "Cabernet Sauvignon Château Montelena Napa Valley 2017"
            },
            {
                "course": "Vegetarian",
                "description": "Daikon and carrot terrine with herbed tofu cream and ponzu sauce",
                "price": 45,
                "wine_pairing": "Sancerre Domaine Vacheron 2020"
            },
            {
                "course": "Cheese",
                "description": "Selection of French and Japanese artisanal cheeses with fruit compote and rice crackers",
                "price": 35,
                "wine_pairing": "Chenin Blanc Domaine Huet Vouvray Le Mont Demi-Sec 2019"
            },
            {
                "course": "Dessert",
                "description": "Green tea crème brûlée with candied yuzu peel and almond sable",
                "price": 25,
                "wine_pairing": "Sauternes Château Rieussec 2017"
            }
        ],
        "grand_total": 395,
        "dining_room_description": "Le Jardin Zen combines the tranquility of a Japanese garden with the sophistication of Parisian flair. The dining room is adorned with delicate cherry blossoms and intricate French-inspired wall art. The gentle murmur of a nearby koi pond, combined with the warm ambient lighting, creates a serene and intimate atmosphere for guests to immerse themselves in the captivating fusion of French and Japanese culinary arts."
    },

    {
        "location": "Four Seasons Hotel Mexico City, Mexico",
        "restaurant_name": "Cielo Azul",
        "chef_name": "Chef Alejandro Torres",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Mini blue corn sope with shredded beef and avocado cream",
                "price": 12,
                "wine_pairing": "Cava Codorníu Gran Plus Ultra Brut Reserva"
            },
            {
                "course": "Appetizer",
                "description": "Trio of ceviches: shrimp, tuna, and octopus, with avocado, serrano, and lime",
                "price": 30,
                "wine_pairing": "Albariño Pazo Señorans 2020"
            },
            {
                "course": "Soup",
                "description": "Roasted tomato and chili soup with crispy tortilla strips and queso fresco",
                "price": 20,
                "wine_pairing": "Sauvignon Blanc Casa Marin Cipreses Vineyard 2019"
            },
            {
                "course": "Fish",
                "description": "Grilled red snapper with mango-pineapple salsa, black bean purée, and coconut rice",
                "price": 40,
                "wine_pairing": "Chardonnay Flowers Sonoma Coast 2018"
            },
            {
                "course": "Meat",
                "description": "Slow-cooked lamb barbacoa with guajillo sauce, nopales, and handmade tortillas",
                "price": 45,
                "wine_pairing": "Tempranillo Muga Reserva 2017"
            },
            {
                "course": "Cheese",
                "description": "Selection of Mexican artisanal cheeses with jalapeño jam and corn crackers",
                "price": 25,
                "wine_pairing": "Malbec Catena Zapata Nicasia Vineyard 2016"
            },
            {
                "course": "Dessert",
                "description": "Chocolate tamal with cinnamon-spiced caramel and vanilla ice cream",
                "price": 18,
                "wine_pairing": "Pedro Ximénez Emilio Hidalgo Gobernador"
            }
        ],
        "grand_total": 190,
        "dining_room_description": "Cielo Azul invites you to experience the vibrant colors and flavors of Mexican cuisine in an elegant and modern setting. The dining room features a breathtaking mural of a blue sky with golden sunrays, evoking the warmth and beauty of Mexico. Handcrafted wooden furniture, traditional pottery, and the soft glow of candlelight create an intimate and welcoming atmosphere, where guests can savor the diverse and delectable tastes of Mexico."
    },

     {
        "location": "Four Seasons Hotel Washington, D.C., USA",
        "restaurant_name": "The American Tapestry",
        "chef_name": "Chef Samuel Hughes",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Maryland crab cake with remoulade sauce",
                "price": 18,
                "wine_pairing": "Chardonnay Sonoma-Cutrer Russian River Ranches 2019"
            },
            {
                "course": "Appetizer",
                "description": "Louisiana gumbo with chicken, andouille sausage, and okra",
                "price": 25,
                "wine_pairing": "Zinfandel Ridge Vineyards East Bench 2018"
            },
            {
                "course": "Salad",
                "description": "California Cobb salad with avocado, bacon, blue cheese, and grilled chicken",
                "price": 22,
                "wine_pairing": "Sauvignon Blanc Kim Crawford Signature Reserve 2020"
            },
            {
                "course": "Fish",
                "description": "New England clam chowder with sourdough bread",
                "price": 28,
                "wine_pairing": "Pinot Gris Willamette Valley Vineyards 2020"
            },
            {
                "course": "Meat",
                "description": "Texas smoked brisket with collard greens and cornbread",
                "price": 45,
                "wine_pairing": "Cabernet Sauvignon Silver Oak Alexander Valley 2016"
            },
            {
                "course": "Vegetarian",
                "description": "Georgia peach and pecan salad with arugula, goat cheese, and honey-mustard vinaigrette",
                "price": 20,
                "wine_pairing": "Riesling Chateau Ste. Michelle Eroica 2019"
            },
            {
                "course": "Dessert",
                "description": "New York cheesecake with strawberry coulis",
                "price": 15,
                "wine_pairing": "Brachetto d'Acqui Marenco Pineto 2020"
            }
        ],
        "grand_total": 173,
        "dining_room_description": "The American Tapestry celebrates the rich culinary traditions of the United States in an elegant and historic setting. Located in the heart of Washington, D.C., the restaurant showcases classic dishes from various regions, capturing the essence of American cuisine. With warm lighting, vintage photographs, and antique maps adorning the walls, guests are invited to embark on a gastronomic journey that pays homage to the diverse flavors and heritage of the nation."
    },

    {
        "location": "Four Seasons Hotel Barcelona, Spain",
        "restaurant_name": "Lab de Sabores",
        "chef_name": "Chef Francisco Gomez",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Mango and passion fruit caviar with mint foam",
                "price": 20,
                "wine_pairing": "Cava Gramona III Lustros Brut Nature 2012"
            },
            {
                "course": "Appetizer",
                "description": "Deconstructed Spanish tortilla with potato espuma, onion gel, and egg yolk spheres",
                "price": 35,
                "wine_pairing": "Verdejo Bodegas Naia Naiades 2017"
            },
            {
                "course": "Fish",
                "description": "Sous-vide sea bass with fennel purée, dehydrated olives, and saffron air",
                "price": 50,
                "wine_pairing": "Albariño Pazo Barrantes 2020"
            },
            {
                "course": "Meat",
                "description": "48-hour braised Iberian pork cheeks with porcini mushroom powder and red wine reduction",
                "price": 60,
                "wine_pairing": "Rioja Roda Reserva 2016"
            },
            {
                "course": "Intermezzo",
                "description": "Mojito sorbet with liquid nitrogen and mint sugar",
                "price": 18,
                "wine_pairing": "None"
            },
            {
                "course": "Cheese",
                "description": "Manchego foam with quince gel and Marcona almond dust",
                "price": 28,
                "wine_pairing": "Sherry Lustau East India Solera"
            },
            {
                "course": "Dessert",
                "description": "Chocolate textures: frozen chocolate mousse, chocolate soil, and warm chocolate foam",
                "price": 30,
                "wine_pairing": "Pedro Ximénez Bodegas Toro Albala Don PX Gran Reserva 1987"
            }
        ],
        "grand_total": 241,
        "dining_room_description": "Lab de Sabores invites guests into a world of culinary innovation, where art meets science in a sleek, modern dining space. Glass walls offer a glimpse into the bustling kitchen, where Chef Gomez and his team create their avant-garde masterpieces. Suspended geometric light fixtures and clean lines evoke a sense of sophistication and futuristic elegance, setting the stage for a truly unique gastronomic experience."
    },

    {
        "location": "Four Seasons Resort Maui, Hawaii, USA",
        "restaurant_name": "Kai 'Olu",
        "chef_name": "Chef Kekoa Mahi",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Ahi poke with avocado mousse and taro chip",
                "price": 30,
                "wine_pairing": "Rosé Domaine Tempier Bandol 2020"
            },
            {
                "course": "Appetizer",
                "description": "Kona lobster tail with coconut milk foam, hearts of palm, and lilikoi vinaigrette",
                "price": 70,
                "wine_pairing": "Sancerre Domaine Vacheron Belle Dame 2019"
            },
            {
                "course": "Soup",
                "description": "Chilled cucumber gazpacho with Kauai shrimp ceviche and pickled mango",
                "price": 35,
                "wine_pairing": "Riesling Dr. Loosen Erdener Prälat Alte Reben 2018"
            },
            {
                "course": "Fish",
                "description": "Pan-seared opah with sweet potato purée, bok choy, and macadamia nut brown butter",
                "price": 85,
                "wine_pairing": "Chardonnay Kistler Les Noisetiers 2019"
            },
            {
                "course": "Meat",
                "description": "Kalua pork with pineapple relish, purple sweet potato mash, and ginger-guava glaze",
                "price": 95,
                "wine_pairing": "Pinot Noir Domaine Serene Yamhill Cuvée 2018"
            },
            {
                "course": "Salad",
                "description": "Maui onion, watermelon radish, and baby greens salad with macadamia nut vinaigrette",
                "price": 40,
                "wine_pairing": "Sauvignon Blanc Cloudy Bay Te Koko 2017"
            },
            {
                "course": "Dessert",
                "description": "Hawaiian chocolate lava cake with passion fruit coulis and macadamia nut ice cream",
                "price": 45,
                "wine_pairing": "Muscat de Rivesaltes Domaine Cazes 2019"
            }
        ],
        "grand_total": 400,
        "dining_room_description": "Kai 'Olu offers an exquisite dining experience that pays homage to Hawaii's rich culinary heritage, with breathtaking views of the Pacific Ocean and the lush gardens of the Four Seasons Resort. The dining room is adorned with warm woods, vibrant textiles, and intricate artwork inspired by traditional Hawaiian motifs. Soft, ambient lighting and the gentle sound of ocean waves create a serene and intimate atmosphere, allowing guests to savor the flavors of the islands in a luxurious and elegant setting."
    },

    {
        "location": "Four Seasons The Nam Hai, Hoi An, Vietnam",
        "restaurant_name": "Sen Vàng",
        "chef_name": "Chef Minh Truong",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Bánh bèo - Steamed rice cakes with shrimp and crispy shallots",
                "price": 15,
                "wine_pairing": "Champagne Billecart-Salmon Brut Réserve"
            },
            {
                "course": "Appetizer",
                "description": "Gỏi cuốn - Fresh spring rolls with prawns, herbs, and peanut dipping sauce",
                "price": 25,
                "wine_pairing": "Gewürztraminer Trimbach 2018"
            },
            {
                "course": "Soup",
                "description": "Phở tái - Beef pho with rare filet mignon, rice noodles, and aromatic herbs",
                "price": 30,
                "wine_pairing": "Pinot Noir Domaine Drouhin Oregon 2018"
            },
            {
                "course": "Fish",
                "description": "Cá kho tộ - Caramelized claypot fish with ginger, chili, and steamed jasmine rice",
                "price": 45,
                "wine_pairing": "Chenin Blanc Domaine Huet Le Mont Sec 2019"
            },
            {
                "course": "Meat",
                "description": "Bò lúc lắc - Shaking beef with garlic, watercress, and pickled onions",
                "price": 50,
                "wine_pairing": "Syrah E. Guigal Crozes-Hermitage 2017"
            },
            {
                "course": "Vegetarian",
                "description": "Bánh xèo - Crispy Vietnamese crepe with tofu, bean sprouts, and nuoc cham",
                "price": 35,
                "wine_pairing": "Riesling Dr. Loosen Blue Slate Kabinett 2020"
            },
            {
                "course": "Dessert",
                "description": "Chè chuối - Banana and tapioca pudding with coconut milk and toasted peanuts",
                "price": 20,
                "wine_pairing": "Sauternes Château Guiraud 2017"
            }
        ],
        "grand_total": 220,
        "dining_room_description": "Sen Vàng offers an elevated dining experience that showcases the vibrant flavors and rich culinary traditions of Vietnam. The elegant dining room is adorned with silk lanterns, intricate wood carvings, and a stunning view of the resort's tropical gardens and reflecting pools. The harmonious blend of traditional and contemporary design elements creates an ambiance of refined sophistication, providing the perfect setting for guests to savor the essence of Vietnamese cuisine."
    },

     {
        "location": "Four Seasons Hotel Mumbai, India",
        "restaurant_name": "Sarvottam",
        "chef_name": "Chef Ravi Desai",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Cashew cheese and beetroot tartlet with mint chutney",
                "price": 20,
                "wine_pairing": "Champagne Drappier Brut Nature Zero Dosage"
            },
            {
                "course": "Appetizer",
                "description": "Crispy chickpea flour pancakes with spiced potato filling and tamarind sauce",
                "price": 30,
                "wine_pairing": "Riesling Trimbach Cuvée Frédéric Émile 2016"
            },
            {
                "course": "Soup",
                "description": "Creamy tomato and coconut soup with curry leaf and toasted almonds",
                "price": 25,
                "wine_pairing": "Sauvignon Blanc Cloudy Bay 2020"
            },
            {
                "course": "Main Course 1",
                "description": "Jackfruit biryani with saffron-infused basmati rice and caramelized onions",
                "price": 40,
                "wine_pairing": "Pinot Gris Domaine Zind-Humbrecht Clos Windsbuhl 2018"
            },
            {
                "course": "Main Course 2",
                "description": "Tandoori cauliflower steak with mint yogurt sauce and quinoa pilaf",
                "price": 45,
                "wine_pairing": "Chardonnay Domaine Laroche Les Vaudevey 2018"
            },
            {
                "course": "Salad",
                "description": "Kale and roasted sweet potato salad with pomegranate seeds and tahini dressing",
                "price": 25,
                "wine_pairing": "Rosé Domaine Tempier Bandol 2020"
            },
            {
                "course": "Dessert",
                "description": "Coconut and cardamom rice pudding with mango coulis and candied pistachios",
                "price": 35,
                "wine_pairing": "Sauternes Château Suduiraut 2017"
            }
        ],
        "grand_total": 220,
        "dining_room_description": "Sarvottam transports guests on a gastronomic journey through the rich and diverse flavors of India, reimagining traditional dishes with a vegan, haute cuisine approach. The luxurious dining room features intricate Jaipuri block-printed textiles, handcrafted brass chandeliers, and opulent marble flooring. Floor-to-ceiling windows offer breathtaking views of the Arabian Sea, providing the perfect backdrop for a truly unforgettable dining experience."
    },

    {
        "location": "Palais Royale, Paris, France",
        "restaurant_name": "Le Jardin Végétal",
        "chef_name": "Chef Amélie Dubois",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Mushroom and truffle pâté with toasted brioche and microgreens",
                "price": 25,
                "wine_pairing": "Champagne Ruinart Blanc de Blancs Brut"
            },
            {
                "course": "Appetizer",
                "description": "Heirloom tomato and vegan mozzarella salad with basil foam and aged balsamic",
                "price": 35,
                "wine_pairing": "Sancerre Domaine Vacheron 2020"
            },
            {
                "course": "Soup",
                "description": "Velouté of green asparagus with almond cream and crispy shallots",
                "price": 30,
                "wine_pairing": "Chardonnay Olivier Leflaive Bourgogne 2019"
            },
            {
                "course": "Main Course 1",
                "description": "Artichoke barigoule with saffron quinoa and olive tapenade",
                "price": 60,
                "wine_pairing": "Meursault Les Narvaux Domaine Michelot 2018"
            },
            {
                "course": "Main Course 2",
                "description": "Vegan cassoulet with smoked tempeh, white beans, and fresh thyme",
                "price": 65,
                "wine_pairing": "Pomerol Château La Pointe 2016"
            },
            {
                "course": "Salad",
                "description": "Young spinach, shaved fennel, and roasted hazelnuts with champagne vinaigrette",
                "price": 30,
                "wine_pairing": "Chablis 1er Cru Montée de Tonnerre Domaine Servin 2019"
            },
            {
                "course": "Dessert",
                "description": "Vegan mille-feuille with raspberry compote and vanilla bean crème pâtissière",
                "price": 40,
                "wine_pairing": "Sauternes Château Lafaurie-Peyraguey 2017"
            }
        ],
        "grand_total": 285,
        "dining_room_description": "Nestled within the historic Palais Royale, Le Jardin Végétal offers a luxurious vegan dining experience that reinterprets classic French cuisine. The opulent dining room features ornate chandeliers, lavish silk draperies, and intricate gold accents. Floor-to-ceiling windows overlook the palace gardens, while the attentive staff ensures an unforgettable evening of gastronomic delights."
    },

    {
        "location": "Tokyo, Japan",
        "restaurant_name": "Sushi Zenkai",
        "chef_name": "Chef Takashi Saito",
        "tasting_menu": [
            {
                "course": "Appetizer",
                "description": "Seasonal vegetable and seaweed salad with yuzu dressing",
                "price": 35,
                "wine_pairing": "Dassai 23 Junmai Daiginjo"
            },
            {
                "course": "Sashimi",
                "description": "Chef's selection of five types of sashimi, including toro, yellowtail, and sea urchin",
                "price": 80,
                "wine_pairing": "Hakkaisan Tokubetsu Honjozo"
            },
            {
                "course": "Nigiri 1",
                "description": "Akami tuna nigiri with freshly grated wasabi",
                "price": 25,
                "wine_pairing": "Kubota Senju Ginjo"
            },
            {
                "course": "Nigiri 2",
                "description": "Kohada (gizzard shad) nigiri with housemade soy glaze",
                "price": 25,
                "wine_pairing": "Kubota Senju Ginjo"
            },
            {
                "course": "Nigiri 3",
                "description": "Botan ebi (sweet shrimp) nigiri with citrus-infused soy sauce",
                "price": 30,
                "wine_pairing": "Kubota Senju Ginjo"
            },
            {
                "course": "Nigiri 4",
                "description": "Japanese scallop nigiri with sea salt and yuzu zest",
                "price": 30,
                "wine_pairing": "Kubota Senju Ginjo"
            },
            {
                "course": "Nigiri 5",
                "description": "A5 Wagyu beef nigiri with truffle shavings and black garlic sauce",
                "price": 50,
                "wine_pairing": "Kubota Senju Ginjo"
            },
            {
                "course": "Temaki",
                "description": "Ikura (salmon roe) temaki with shiso leaf and crispy nori",
                "price": 40,
                "wine_pairing": "Dewazakura Oka Ginjo"
            },
            {
                "course": "Maki",
                "description": "Futomaki with kanpyo, tamago, and pickled vegetables",
                "price": 30,
                "wine_pairing": "Dewazakura Oka Ginjo"
            },
            {
                "course": "Dessert",
                "description": "Yuzu sorbet with fresh seasonal fruit and matcha dust",
                "price": 20,
                "wine_pairing": "Hakushika Hanaomoi Junmai Ginjo"
            }
        ],
        "grand_total": 335,
        "dining_room_description": "Sushi Zenkai offers an extraordinary Omakase experience, led by world-renowned Chef Takashi Saito. The intimate, minimalist dining room features a beautiful hinoki wood sushi counter, where guests can watch the masterful craftsmanship of the chef up close. The attentive staff ensures an unparalleled sushi experience that will leave guests in awe."
    },

    {
        "location": "Four Seasons Mountain Retreat, Swiss Alps",
        "restaurant_name": "Alpine Heights",
        "chef_name": "Chef Lukas Baumgartner",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Mini raclette toast with cornichon and pearl onion",
                "price": 20,
                "wine_pairing": "Champagne Ruinart Blanc de Blancs Brut"
            },
            {
                "course": "Appetizer",
                "description": "Swiss chard and gruyère tart with a walnut crust",
                "price": 35,
                "wine_pairing": "Fendant Clos de la Morge Domaine Rouvinez 2019"
            },
            {
                "course": "Soup",
                "description": "Creamy leek and potato soup with crispy rösti croutons",
                "price": 30,
                "wine_pairing": "Sauvignon Blanc Domaine Jean-René Germanier 2020"
            },
            {
                "course": "Fish",
                "description": "Arctic char with brown butter, capers, and roasted root vegetables",
                "price": 55,
                "wine_pairing": "Chasselas Domaine de Autecour 2018"
            },
            {
                "course": "Meat",
                "description": "Slow-braised venison in red wine sauce with spätzle and braised red cabbage",
                "price": 65,
                "wine_pairing": "Pinot Noir Domaine Jean-René Germanier Cayas 2017"
            },
            {
                "course": "Cheese",
                "description": "Selection of Swiss cheeses with fruit compote and artisanal bread",
                "price": 40,
                "wine_pairing": "Humagne Rouge Domaine de Beudon 2018"
            },
            {
                "course": "Dessert",
                "description": "Dark chocolate fondue with fresh fruits and homemade marshmallows",
                "price": 25,
                "wine_pairing": "Eiswein Domaine des Muses 2017"
            }
        ],
        "grand_total": 270,
        "dining_room_description": "Perched atop the Swiss Alps, Alpine Heights offers a breathtaking dining experience with panoramic views of snow-capped peaks and pristine valleys. The cozy and intimate dining room features rustic timber beams, a roaring fireplace, and plush velvet seating. The exquisite menu showcases the best of Swiss cuisine, elevated by modern techniques and seasonal, locally sourced ingredients."
    },
      {
        "location": "Four Seasons Hotel, New New York, Futurama",
        "restaurant_name": "Galactic Delights",
        "chef_name": "Chef Zilgax-5",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Spherified root beer float with molecularly-stabilized whipped cream",
                "price": 100,
                "wine_pairing": "Gamma-7 Sparkling Nebula Wine"
            },
            {
                "course": "Appetizer",
                "description": "Microgravity-grown salad with asteroidal vinaigrette and solar-dried tomatoes",
                "price": 150,
                "wine_pairing": "Proxima Centauri Chardonnay 2998"
            },
            {
                "course": "Soup",
                "description": "Cream of Martian potato soup with levitating croutons",
                "price": 120,
                "wine_pairing": "Algol Binary Star Sauvignon Blanc 2999"
            },
            {
                "course": "Fish",
                "description": "Pan-seared Venusian mackerel with lunar seaweed and comet dust",
                "price": 200,
                "wine_pairing": "Betelgeuse Supernova Riesling 2997"
            },
            {
                "course": "Meat",
                "description": "Roasted Plutonian quail with Andromeda asparagus and wormhole truffle sauce",
                "price": 250,
                "wine_pairing": "Kepler-186f Pinot Noir 2996"
            },
            {
                "course": "Cheese",
                "description": "Selection of artisanal cheeses from the Milky Way with stardust crackers",
                "price": 180,
                "wine_pairing": "Gliese 581g Merlot 2995"
            },
            {
                "course": "Dessert",
                "description": "Dark matter chocolate mousse with freeze-dried raspberry coulis and supernova sugar",
                "price": 150,
                "wine_pairing": "Sirius Star System Ice Wine 2994"
            }
        ],
        "grand_total": 1050,
        "dining_room_description": "Galactic Delights offers an out-of-this-world dining experience, set in the heart of New New York's Futurama. The sleek and modern dining room boasts panoramic views of the city's futuristic skyline, with hovering vehicles and interstellar traffic visible through the floor-to-ceiling windows. The menu showcases innovative fusion dishes, utilizing exotic ingredients sourced from across the universe, prepared with cutting-edge techniques that blur the line between science and culinary art."
    },
    {
        "location": "Four Seasons Casino, South of France",
        "restaurant_name": "Le Festin Opulent",
        "chef_name": "Chef Antoine Dubois",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "24-karat gold leaf foie gras torchon with black truffle shavings",
                "price": 150,
                "wine_pairing": "Dom Pérignon Vintage 2015"
            },
            {
                "course": "Appetizer",
                "description": "Caviar-topped oysters with champagne sabayon and edible pearls",
                "price": 300,
                "wine_pairing": "Krug Clos du Mesnil Blanc de Blancs 2013"
            },
            {
                "course": "Soup",
                "description": "Lobster bisque with cognac and saffron cream",
                "price": 200,
                "wine_pairing": "Château d'Yquem Sauternes 2018"
            },
            {
                "course": "Fish",
                "description": "Seared wild turbot with Beluga lentils, white asparagus, and caviar beurre blanc",
                "price": 350,
                "wine_pairing": "Château Margaux Blanc 2016"
            },
            {
                "course": "Meat",
                "description": "Wagyu beef Rossini with black truffle, seared foie gras, and Madeira reduction",
                "price": 500,
                "wine_pairing": "Château Pétrus Pomerol 2014"
            },
            {
                "course": "Cheese",
                "description": "Assortment of rare and aged French cheeses with truffle honey and gold-flecked crackers",
                "price": 250,
                "wine_pairing": "Château Mouton Rothschild Pauillac 2015"
            },
            {
                "course": "Dessert",
                "description": "Louis XV chocolate mousse cake with gold leaf and candied violets",
                "price": 200,
                "wine_pairing": "Château Climens Barsac 2017"
            }
        ],
        "grand_total": 1950,
        "dining_room_description": "Le Festin Opulent offers a dining experience fit for royalty in the heart of the South of France's most glamorous casino. The elegant dining room is adorned with gold accents, shimmering chandeliers, and opulent silk drapery. The expansive floor-to-ceiling windows offer breathtaking views of the Mediterranean coastline, while the indulgent menu showcases the most exquisite and luxurious ingredients, masterfully crafted into a truly decadent feast."
    },

    {
        "location": "Four Seasons Mykonos, Greece",
        "restaurant_name": "The Golden Aegean",
        "chef_name": "Chef Stavros Papadopoulos",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Filo-wrapped Kalamata olives with gold leaf, served on a bed of ouzo-infused tomato jam",
                "price": 100,
                "wine_pairing": "Taittinger Comtes de Champagne Blanc de Blancs 2010"
            },
            {
                "course": "Appetizer",
                "description": "Duo of seared scallops and tiger prawns with Greek saffron and white truffle aioli",
                "price": 200,
                "wine_pairing": "Domaine Sigalas Assyrtiko Santorini 2020"
            },
            {
                "course": "Salad",
                "description": "Golden beet and goat cheese salad with pomegranate molasses and 24-karat gold leaf",
                "price": 150,
                "wine_pairing": "Alpha Estate Axia Malagousia 2019"
            },
            {
                "course": "Fish",
                "description": "Grilled wild sea bass with Aegean sea urchin butter, fennel confit, and champagne foam",
                "price": 300,
                "wine_pairing": "Domaine Gerovassiliou Viognier Epanomi 2019"
            },
            {
                "course": "Meat",
                "description": "Aged lamb rack with Mykonian honey glaze, black truffle moussaka, and gold leaf-flecked orzo",
                "price": 400,
                "wine_pairing": "Kir-Yianni Ramnista Xinomavro Naoussa 2016"
            },
            {
                "course": "Cheese",
                "description": "Assortment of premium Greek cheeses with fig preserves, gold-dusted almonds, and honeycomb",
                "price": 250,
                "wine_pairing": "Ktima Pavlidis Thema Red Drama 2017"
            },
            {
                "course": "Dessert",
                "description": "Mastiha-infused chocolate mousse with gold leaf and rosewater-scented baklava crumble",
                "price": 150,
                "wine_pairing": "Muscat of Samos Nectar 2017"
            }
        ],
        "grand_total": 1550,
        "dining_room_description": "The Golden Aegean offers an unparalleled dining experience, combining the finest ingredients from the Greek islands with the most luxurious and decadent touches. The elegant dining room features stunning panoramic views of the Aegean Sea, while the outdoor terrace allows guests to dine under the stars. The menu showcases the best of traditional Greek cuisine, reimagined with an opulent twist, for an unforgettable gastronomic experience."
    },

    {
        "location": "Four Seasons Palazzo Siciliano, Sicily",
        "restaurant_name": "La Cucina dei Sogni",
        "chef_name": "Chef Alessandro De Angelis",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Crispy arancini with wild mushroom and black truffle filling",
                "price": 75,
                "wine_pairing": "Franciacorta Ca' del Bosco Cuvée Annamaria Clementi 2010"
            },
            {
                "course": "Appetizer",
                "description": "Seared foie gras with fig balsamic glaze and aged Parmigiano-Reggiano crisp",
                "price": 150,
                "wine_pairing": "Barolo Gaja Sperss 2016"
            },
            {
                "course": "Pasta",
                "description": "Handmade pappardelle with slow-cooked wild boar ragù and shaved Pecorino Romano",
                "price": 100,
                "wine_pairing": "Brunello di Montalcino Biondi Santi Riserva 2013"
            },
            {
                "course": "Fish",
                "description": "Grilled Mediterranean langoustines with saffron risotto and lobster bisque reduction",
                "price": 200,
                "wine_pairing": "Sassicaia Tenuta San Guido 2018"
            },
            {
                "course": "Meat",
                "description": "Osso buco with gremolata, truffle-infused polenta, and Barolo demi-glace",
                "price": 250,
                "wine_pairing": "Amarone della Valpolicella Quintarelli Riserva 2009"
            },
            {
                "course": "Cheese",
                "description": "Selection of fine Italian cheeses with fruit compote, artisanal bread, and balsamic pearls",
                "price": 100,
                "wine_pairing": "Tignanello Antinori 2017"
            },
            {
                "course": "Dessert",
                "description": "Tiramisu al limoncello with candied lemon zest and pistachio crumble",
                "price": 75,
                "wine_pairing": "Moscato d'Asti Vietti Cascinetta 2020"
            }
        ],
        "grand_total": 950,
        "dining_room_description": "La Cucina dei Sogni welcomes guests to a world of unparalleled Italian luxury, set within the majestic confines of a historic Sicilian palazzo. The opulent dining room is adorned with frescoes, marble floors, and crystal chandeliers, providing an elegant and romantic atmosphere. The menu, curated by renowned Chef Alessandro De Angelis, celebrates the finest Italian ingredients and traditional dishes, while an expert sommelier ensures the perfect wine pairings for a truly unforgettable dining experience."
    },

    {
        "location": "Four Seasons Skyscraper, New York City",
        "restaurant_name": "Parkview Elegance",
        "chef_name": "Chef Thomas Kingsley",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Kumamoto oyster with cucumber mignonette and smoked caviar",
                "price": 85,
                "wine_pairing": "Dom Pérignon Vintage 2010"
            },
            {
                "course": "Appetizer",
                "description": "Butter-poached Maine lobster with parsnip purée, microgreens, and saffron beurre blanc",
                "price": 175,
                "wine_pairing": "Chardonnay Peter Michael Ma Belle-Fille 2018"
            },
            {
                "course": "Soup",
                "description": "Creamy wild mushroom bisque with black truffle foam and brioche croutons",
                "price": 100,
                "wine_pairing": "Pinot Noir Kosta Browne Russian River Valley 2018"
            },
            {
                "course": "Fish",
                "description": "Seared diver scallops with cauliflower purée, caviar, and Champagne beurre blanc",
                "price": 150,
                "wine_pairing": "Puligny-Montrachet Domaine Leflaive Les Pucelles 2016"
            },
            {
                "course": "Meat",
                "description": "Wagyu beef tenderloin with potato gratin, foie gras, and Perigord truffle jus",
                "price": 350,
                "wine_pairing": "Cabernet Sauvignon Harlan Estate 2014"
            },
            {
                "course": "Cheese",
                "description": "Selection of artisanal cheeses with fruit preserves, Marcona almonds, and walnut-raisin bread",
                "price": 85,
                "wine_pairing": "Château d'Yquem Sauternes 2009"
            },
            {
                "course": "Dessert",
                "description": "Dark chocolate soufflé with passion fruit coulis and Tahitian vanilla ice cream",
                "price": 75,
                "wine_pairing": "Taylor Fladgate 20 Year Old Tawny Port"
            }
        ],
        "grand_total": 1020,
        "dining_room_description": "Parkview Elegance offers an exquisite dining experience in a sophisticated setting, high above the city on the 55th floor of the Four Seasons Skyscraper. The floor-to-ceiling windows provide breathtaking panoramic views of Central Park, while the opulent dining room features plush velvet seating, crystal chandeliers, and fine art. The menu, crafted by Chef Thomas Kingsley, showcases the finest ingredients and innovative techniques, paired with a remarkable wine list for an unforgettable culinary journey."
    },

    {
        "location": "Four Seasons Ice and Geyser Palace, Iceland",
        "restaurant_name": "Aurora Borealis",
        "chef_name": "Chef Hyeon Min-jun",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Seared Arctic char tartare with gochujang aioli and crispy seaweed",
                "price": 30,
                "wine_pairing": "Champagne Billecart-Salmon Brut Réserve"
            },
            {
                "course": "Appetizer",
                "description": "Icelandic lamb mandu with fermented cabbage and doenjang dipping sauce",
                "price": 50,
                "wine_pairing": "Riesling Egon Müller Scharzhofberger Kabinett 2019"
            },
            {
                "course": "Soup",
                "description": "Miso and dulse seaweed soup with smoked tofu and wild mushrooms",
                "price": 35,
                "wine_pairing": "Gruner Veltliner Hirsch Kammern Heiligenstein 2018"
            },
            {
                "course": "Fish",
                "description": "Pan-seared Atlantic cod with kimchi fried rice and pickled radish",
                "price": 65,
                "wine_pairing": "Chardonnay Kistler Vine Hill Vineyard 2017"
            },
            {
                "course": "Meat",
                "description": "Bulgogi-marinated reindeer with Icelandic barley risotto and ssamjang sauce",
                "price": 85,
                "wine_pairing": "Pinot Noir Domaine de la Romanée-Conti Echézeaux 2015"
            },
            {
                "course": "Dessert",
                "description": "Yuzu-infused Skyr mousse with honeycomb and mixed berry coulis",
                "price": 45,
                "wine_pairing": "Tokaji Aszú 6 Puttonyos Oremus 2013"
            },
            {
                "course": "Petit Fours",
                "description": "Matcha-dusted Icelandic chocolate truffles and sesame-honey brittle",
                "price": 25,
                "wine_pairing": "Sauternes Château Coutet 2016"
            }
        ],
        "grand_total": 335,
        "dining_room_description": "Aurora Borealis is a breathtaking dining experience that combines the beauty of Icelandic nature with the bold flavors of Korean cuisine. Set within the enchanting Four Seasons Ice and Geyser Palace, the restaurant features a magnificent ice-carved chandelier, floor-to-ceiling windows overlooking geysers, and plush seating designed to reflect the Northern Lights. Chef Hyeon Min-jun masterfully blends the finest local ingredients with Korean culinary techniques for an unforgettable fusion feast."
    },

    {
        "location": "Four Seasons Jungle Retreat, South America",
        "restaurant_name": "Pescado Sagrado",
        "chef_name": "Chef Alejandro Torres",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Wild-caught Amazonian shrimp ceviche with lime, mango, and aji amarillo",
                "price": 35,
                "wine_pairing": "Champagne Dom Pérignon Vintage 2010"
            },
            {
                "course": "Appetizer",
                "description": "Tiradito of sustainably caught sea bass with avocado mousse and passion fruit vinaigrette",
                "price": 50,
                "wine_pairing": "Albariño Pazo Señorans Selección de Añada 2015"
            },
            {
                "course": "Salad",
                "description": "Heart of palm and quinoa salad with pomegranate seeds and chia-lime dressing",
                "price": 40,
                "wine_pairing": "Sauvignon Blanc Cloudy Bay Te Koko 2017"
            },
            {
                "course": "Fish",
                "description": "Cured Patagonian toothfish with chile-infused tapioca pearls and tamarillo sauce",
                "price": 65,
                "wine_pairing": "Chardonnay Catena Zapata Adrianna Vineyard White Bones 2018"
            },
            {
                "course": "Main Course",
                "description": "Amazonian catfish sashimi with heirloom tomato salsa and a side of purple corn arepas",
                "price": 75,
                "wine_pairing": "Chenin Blanc François Chidaine Montlouis-sur-Loire Les Bournais 2016"
            },
            {
                "course": "Pre-Dessert",
                "description": "Acai sorbet with cacao nibs and an edible flower garnish",
                "price": 25,
                "wine_pairing": "Moscato d'Asti Elio Perrone Sourgal 2020"
            },
            {
                "course": "Dessert",
                "description": "Grilled pineapple carpaccio with coconut foam and guava coulis",
                "price": 45,
                "wine_pairing": "Sauternes Château d'Yquem 2011"
            }
        ],
        "grand_total": 335,
        "dining_room_description": "Pescado Sagrado offers an unparalleled dining experience in the heart of the South American jungle, accessible only by helicopter. This exclusive Four Seasons Jungle Retreat restaurant blends modern culinary techniques with the rich history and flavors of indigenous culture. The elegantly designed dining room features floor-to-ceiling windows that immerse guests in the lush jungle surroundings, while the open-air terrace provides a unique opportunity to dine under the stars."
    },

    {
        "location": "Four Seasons Hotel, San Francisco",
        "restaurant_name": "Sol y Sakura",
        "chef_name": "Chef Isabella Sánchez",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Tuna and piquillo pepper temaki with shiso and Spanish olive oil",
                "price": 20,
                "wine_pairing": "Cava Recaredo Brut Nature Gran Reserva 2015"
            },
            {
                "course": "Appetizer",
                "description": "Seared scallops with jamón ibérico and yuzu miso sauce",
                "price": 45,
                "wine_pairing": "Albariño Do Ferreiro Cepas Vellas 2019"
            },
            {
                "course": "Soup",
                "description": "Gazpacho with uni, daikon radish, and wasabi oil",
                "price": 30,
                "wine_pairing": "Sancerre Domaine Vacheron Belle Dame 2018"
            },
            {
                "course": "Fish",
                "description": "Grilled octopus with shishito peppers, sake-glazed eggplant, and romesco sauce",
                "price": 60,
                "wine_pairing": "Verdejo Belondrade y Lurton 2018"
            },
            {
                "course": "Meat",
                "description": "Wagyu beef tataki with pimientos de Padrón, garlic chips, and ponzu reduction",
                "price": 70,
                "wine_pairing": "Rioja Roda I Reserva 2012"
            },
            {
                "course": "Rice",
                "description": "Saffron-infused sushi rice with roasted Mediterranean vegetables and crispy nori",
                "price": 40,
                "wine_pairing": "Chardonnay Marimar Estate La Màsía 2018"
            },
            {
                "course": "Dessert",
                "description": "Green tea and pistachio crème brûlée with a sesame brittle",
                "price": 25,
                "wine_pairing": "Pedro Ximénez Toro Albala Don PX Gran Reserva 1987"
            }
        ],
        "grand_total": 290,
        "dining_room_description": "Sol y Sakura presents a stunning fusion of Spanish and Japanese cuisines in the heart of San Francisco at the luxurious Four Seasons Hotel. The elegant dining room showcases breathtaking views of the San Francisco Bay through its expansive windows, while a stylish outdoor terrace allows guests to enjoy the city's renowned skyline. The restaurant's unique menu, designed by acclaimed Chef Isabella Sánchez, offers a harmonious blend of flavors that will transport diners on a remarkable culinary journey."
    },

    {
        "location": "Four Seasons Hotel, Reims",
        "restaurant_name": "Le Papillon d'Or",
        "chef_name": "Chef Claude Martin",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Smoked salmon mousse on blinis with caviar and chives",
                "price": 25,
                "wine_pairing": "Taittinger Brut Réserve"
            },
            {
                "course": "Appetizer",
                "description": "Seared foie gras with poached pear and brioche toast",
                "price": 45,
                "wine_pairing": "Billecart-Salmon Cuvée Nicolas François Billecart 2006"
            },
            {
                "course": "Fish",
                "description": "Sole meunière with brown butter, capers, and lemon",
                "price": 55,
                "wine_pairing": "Louis Roederer Cristal Brut 2012"
            },
            {
                "course": "Meat",
                "description": "Roasted rack of lamb with ratatouille and rosemary jus",
                "price": 65,
                "wine_pairing": "Moët & Chandon Grand Vintage Rosé 2013"
            },
            {
                "course": "Cheese",
                "description": "Selection of regional French cheeses with fruit compote and walnut bread",
                "price": 35,
                "wine_pairing": "Pol Roger Brut Vintage 2012"
            },
            {
                "course": "Dessert",
                "description": "Tarte Tatin with crème fraîche and caramel sauce",
                "price": 30,
                "wine_pairing": "Veuve Clicquot Demi-Sec"
            }
        ],
        "grand_total": 255,
        "dining_room_description": "Le Papillon d'Or is an iconic Four Seasons restaurant located in the heart of Reims, adorned with exquisite Art Nouveau details and luxurious furnishings. The restaurant offers an exceptional dining experience, featuring a menu of traditional French cuisine expertly paired with a selection of both prestigious and affordable champagnes. The elegant atmosphere and extraordinary service make Le Papillon d'Or the perfect destination for a truly unforgettable evening."
    },

    {
        "location": "Four Seasons Hotel, Paris",
        "restaurant_name": "Le Château des Choux",
        "chef_name": "Chef Marie Dubois",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Mini gougères with Comté cheese and fresh herbs",
                "price": 20,
                "wine_pairing": "Champagne Bollinger Special Cuvée Brut"
            },
            {
                "course": "Appetizer",
                "description": "Savory éclair filled with goat cheese mousse, sun-dried tomatoes, and basil",
                "price": 30,
                "wine_pairing": "Sancerre Domaine Vacheron 2020"
            },
            {
                "course": "Fish",
                "description": "Salmon mousse-filled profiteroles with dill cream sauce",
                "price": 40,
                "wine_pairing": "Chablis Domaine William Fèvre 2019"
            },
            {
                "course": "Meat",
                "description": "Savory religieuse with duck confit and foie gras mousse, topped with truffle glaze",
                "price": 50,
                "wine_pairing": "Saint-Émilion Château Figeac 2015"
            },
            {
                "course": "Pre-Dessert",
                "description": "Lemon meringue éclair with tangy lemon curd filling and torched meringue",
                "price": 25,
                "wine_pairing": "Muscat de Beaumes-de-Venise Domaine de Durban 2018"
            },
            {
                "course": "Dessert",
                "description": "Classic chocolate éclair with rich Valrhona chocolate cream and chocolate glaze",
                "price": 25,
                "wine_pairing": "Banyuls Domaine La Tour Vieille 2018"
            },
            {
                "course": "Petit Four",
                "description": "Raspberry and pistachio religieuse with raspberry cream and pistachio mousseline",
                "price": 25,
                "wine_pairing": "Loupiac Château du Cros 2016"
            }
        ],
        "grand_total": 215,
        "dining_room_description": "Le Château des Choux, nestled within a lavish Parisian palace, is a truly unique dining experience that highlights the exquisite art of cream puff pastry. The elegant setting, with its opulent chandeliers and plush velvet seating, provides the perfect backdrop for Chef Marie Dubois's innovative savory and sweet creations. Each dish is thoughtfully paired with a wine from various French regions, showcasing the diverse flavors of the country's rich culinary heritage."
    },

     {
        "location": "Four Seasons Hotel, Malaga, Spain",
        "restaurant_name": "La Esencia",
        "chef_name": "Chef Alejandro Torres",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Jamon Iberico with pan con tomate",
                "price": 25,
                "wine_pairing": "Cava Gramona Imperial Gran Reserva Brut 2016"
            },
            {
                "course": "Appetizer",
                "description": "Salmorejo with quail egg and crispy Serrano ham",
                "price": 30,
                "wine_pairing": "Albariño Pazo de Señoráns 2020"
            },
            {
                "course": "Fish",
                "description": "Grilled octopus with potato cream and smoked paprika",
                "price": 45,
                "wine_pairing": "Godello Bodegas Avancia 2019"
            },
            {
                "course": "Meat",
                "description": "Cordero al Chilindrón – Slow-cooked lamb with red pepper, tomato, and onion sauce",
                "price": 50,
                "wine_pairing": "Rioja Marqués de Riscal Reserva 2015"
            },
            {
                "course": "Cheese",
                "description": "Spanish cheese selection with quince paste, Marcona almonds, and olive oil crackers",
                "price": 35,
                "wine_pairing": "Priorat Clos Mogador 2017"
            },
            {
                "course": "Dessert",
                "description": "Torrijas – Caramelized brioche soaked in cinnamon-infused milk, served with vanilla ice cream",
                "price": 30,
                "wine_pairing": "Pedro Ximénez Bodegas Toro Albala Don PX Gran Reserva 1987"
            }
        ],
        "grand_total": 215,
        "dining_room_description": "La Esencia, located in the heart of Malaga, showcases the rich flavors of Spanish cuisine, combining time-honored recipes with modern culinary techniques. The elegant dining room features warm colors, rustic wooden accents, and vibrant Spanish artwork. The stunning views of the Mediterranean Sea provide a breathtaking backdrop for an unforgettable gastronomic experience, enhanced by a selection of the finest Spanish wines and cava."
    },

    {
        "location": "Four Seasons Hotel London at Ten Trinity Square",
        "restaurant_name": "La Dame de Pic",
        "chef_name": "Chef Anne-Sophie Pic",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Red pepper and raspberry gazpacho with basil and coriander",
                "price": 20,
                "wine_pairing": "Champagne Krug Grande Cuvée"
            },
            {
                "course": "Appetizer",
                "description": "Smoked eel with beetroot, horseradish, and granny smith apple",
                "price": 35,
                "wine_pairing": "Sancerre François Cotat Les Monts Damnés 2019"
            },
            {
                "course": "Fish",
                "description": "Pan-seared scallops with cauliflower, brown butter, and curry leaf",
                "price": 45,
                "wine_pairing": "Chablis Domaine William Fèvre Grand Cru Les Clos 2018"
            },
            {
                "course": "Meat",
                "description": "Venison with chestnuts, juniper berries, and parsnip purée",
                "price": 55,
                "wine_pairing": "Pomerol Château Gazin 2016"
            },
            {
                "course": "Cheese",
                "description": "Selection of French cheeses with fig jam and walnut bread",
                "price": 30,
                "wine_pairing": "Gevrey-Chambertin Domaine Fourrier Vieilles Vignes 2017"
            },
            {
                "course": "Dessert",
                "description": "Chocolate with passion fruit, ginger, and hazelnut crumble",
                "price": 25,
                "wine_pairing": "Banyuls Domaine La Tour Vieille Rimage 2018"
            }
        ],
        "grand_total": 210,
        "dining_room_description": "La Dame de Pic, a Michelin-starred French restaurant, offers an exquisite culinary experience in the heart of London. The elegant dining room features soft lighting, warm colors, and luxurious furnishings, creating a serene atmosphere for guests to enjoy. Chef Anne-Sophie Pic's contemporary cuisine focuses on aromatic herbs and spices, beautifully showcasing her passion for creating memorable, harmonious dishes. Each course is paired with a carefully selected wine, enhancing the flavors and providing a truly unforgettable dining experience."
    },

    {
        "location": "Four Seasons Hotel George V, Paris",
        "restaurant_name": "The Imperial Room",
        "chef_name": "Chef Pierre Gagnaire",
        "tasting_menu": [
            {
                "course": "Amuse-Bouche",
                "description": "Caviar with blinis and crème fraîche",
                "price": 200,
                "wine_pairing": "Dom Pérignon Vintage 2010"
            },
            {
                "course": "Appetizer",
                "description": "Seared scallops with black truffle and Jerusalem artichoke purée",
                "price": 400,
                "wine_pairing": "Meursault Domaine des Comtes Lafon Les Perrières 2018"
            },
            {
                "course": "Soup",
                "description": "Lobster bisque with saffron and cognac",
                "price": 350,
                "wine_pairing": "Château-Grillet 2015"
            },
            {
                "course": "Fish",
                "description": "Turbot with Champagne sauce, caviar, and asparagus",
                "price": 450,
                "wine_pairing": "Corton-Charlemagne Grand Cru Domaine Coche-Dury 2017"
            },
            {
                "course": "Meat",
                "description": "Roasted duck with orange and kumquat, foie gras and black currant sauce",
                "price": 500,
                "wine_pairing": "Château Mouton Rothschild 2009"
            },
            {
                "course": "Cheese",
                "description": "Selection of French cheeses with truffle honey and walnut bread",
                "price": 300,
                "wine_pairing": "Château d'Yquem 1999"
            },
            {
                "course": "Dessert",
                "description": "Crème brûlée with vanilla and raspberry, gold leaf garnish",
                "price": 350,
                "wine_pairing": "Krug Clos du Mesnil 2006"
            }
        ],
        "grand_total": 3500,
        "dining_room_description": "The Imperial Room is a luxurious restaurant located within the Four Seasons Hotel George V in Paris. Known for its opulent décor and historic charm, the restaurant features hand-painted silk walls, ornate plasterwork ceilings, polished wood tables, and plush velvet chairs. A grand chandelier made of crystal and gold serves as the centerpiece of this classic, elegant dining space. The sophisticated menu, created by Chef Pierre Gagnaire, showcases classic French cuisine with a modern twist, using the finest ingredients like caviar, truffles, and foie gras. The impeccable, attentive service and extensive wine list, including rare vintages and exclusive labels, complete the unforgettable dining experience at The Imperial Room."
    }]

"""


## default test call

In [4]:
from pydantic import BaseModel
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

In [5]:
event

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

## test hot sauce prompt

In [6]:
class HotSauce(BaseModel):
    name: str
    bottling_date: str
    hotness_level: int
    description: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an expert in hot sauces."},
        {"role": "user", "content": "Create a new hot sauce."},
    ],
    response_format=HotSauce,
)

event = completion.choices[0].message.parsed

In [7]:
event

HotSauce(name='Fuego de Luna', bottling_date='2023-10-15', hotness_level=8, description='Fuego de Luna is a vibrant hot sauce that captures the fiery essence of the harvest moon. This sauce combines the rich smokiness of fire-roasted habanero peppers with the tropical sweetness of mangos and a subtle hint of lime. Its well-balanced heat and flavor make it perfect for those who seek a bold, spicy experience with a touch of fruitiness. Ideal for tacos, grilled meats, or even to spice up your morning eggs, Fuego de Luna is crafted for those who love their heat with a twist of the exotic.')

## Get the dishes  

We extract the dishes from the large JSON file and keep the course name, description, and wine pairing, for the recipe generation.

In [12]:
import json

json_recipes = json.loads(recipe)

In [21]:
json_recipes[0]['tasting_menu'][0]

{'course': 'Amuse-Bouche',
 'description': 'Crispy Parmesan waffle with tomato and basil gelée',
 'price': 15,
 'wine_pairing': 'Champagne Henriot Brut Souverain'}

In [28]:
dish_str_list = []

for curr_menu in json_recipes:
  for dish in curr_menu['tasting_menu']:
    dish_str = f"""Course: {dish['course']}
Description: {dish['description']}
wine_pairing: {dish['wine_pairing']}
"""
    # print(dish_str)
    dish_str_list.append(dish_str)

## Get the filenames

We also have to get a filename for each dish, so we can name the resulting markdown file.

In [49]:
menu_filenames = []

for curr_menu in json_recipes:
  for dish in curr_menu['tasting_menu']:
    filename = dish['course'].replace(' ', '_') +'-'+ dish['description'].replace(' ', '_') + '.md'

    # print(filename)
    menu_filenames.append(filename)

## test a single recipe markdown  

We opt for a markdown recipe, as it will be more flexible than a global JSON.  
We still use JSON mode to make sure only the recipe is passed down by GPT.

In [29]:
dish_str_list[0]

'Course: Amuse-Bouche \nDescription: Crispy Parmesan waffle with tomato and basil gelée\nwine_pairing: Champagne Henriot Brut Souverain\n'

In [40]:
class Recipe(BaseModel):
    recipe: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a Michelin starred chef. You are given a dish. Provide the recipe in Markdown format."},
        {"role": "user", "content": dish_str_list[0]},
    ],
    response_format=Recipe,
)

recipe = completion.choices[0].message.parsed.recipe
with open(menu_filenames[0], "w") as markdown_file:
    markdown_file.write(recipe)


In [41]:
with open(menu_filenames[0], "r") as markdown_file:
    content = markdown_file.read()

In [42]:
from IPython.display import display_markdown

display_markdown(content, raw=True)


# Crispy Parmesan Waffle with Tomato and Basil Gelée

### Ingredients:

#### For the Parmesan Waffle:
- 1 cup all-purpose flour
- 1/2 cup finely grated Parmesan cheese
- 2 teaspoons baking powder
- 1/2 teaspoon salt
- 1/4 teaspoon black pepper
- 1 cup whole milk
- 2 large eggs
- 4 tablespoons unsalted butter, melted and cooled

#### For the Tomato and Basil Gelée:
- 5 ripe Roma tomatoes, chopped
- 1 clove garlic, minced
- 1/4 cup fresh basil leaves, tightly packed
- 1 tablespoon olive oil
- 1 teaspoon unflavored gelatin
- Salt and pepper, to taste

### Instructions:

#### Making the Parmesan Waffle:
1. **Preheat your Waffle Iron:** Start by preheating your waffle iron according to the manufacturer's instructions.
2. **Prepare the Dry Ingredients:** In a large bowl, whisk together the flour, grated Parmesan cheese, baking powder, salt, and pepper.
3. **Combine Wet Ingredients:** In another bowl, whisk the whole milk, eggs, and melted butter until well blended.
4. **Mix Together:** Pour the wet ingredients into the dry ingredients, mixing gently until just combined. Do not overmix.
5. **Cook the Waffles:** Lightly grease the waffle iron. Spoon the batter into the preheated waffle iron and cook for about 3-4 minutes or until golden brown and crispy. Remove and set aside.

#### Making the Tomato and Basil Gelée:
1. **Prepare the Gelée Base:** In a saucepan over medium heat, add chopped tomatoes and garlic. Cook until tomatoes are soft and release their juices, about 5-7 minutes.
2. **Strain the Mixture:** Remove the tomato mixture from heat and strain through a fine mesh sieve, pressing down to extract as much liquid as possible. Discard the solids.
3. **Bloom the Gelatin:** Sprinkle gelatin over 1 tablespoon of cold water in a small bowl and let stand for about 1-2 minutes to bloom.
4. **Combine & Season:** Return the strained tomato liquid to the pan, stirring in the gelatin until completely dissolved. Add olive oil and season with salt and pepper to taste. Stir in basil leaves.
5. **Chill:** Pour the tomato mixture into a shallow pan and chill in the refrigerator for at least 2 hours, or until set.

### Assemble the Amuse-Bouche:
1. **Cut the Waffles:** Once waffles are cooled slightly, cut them into small, bite-sized squares or circles.
2. **Top with Gelée:** Using a spoon or a small mold, place a piece of tomato and basil gelée on top of each waffle piece.
3. **Presentation:** Arrange on a serving platter garnished with small fresh basil leaves and serve immediately.

### Wine Pairing:
- Serve with a glass of Champagne Henriot Brut Souverain, which beautifully complements the savory richness of the Parmesan and the fresh, vibrant flavors of the tomato and basil.

Enjoy this sophisticated and flavorful amuse-bouche that delights with its light crispy texture and aromatic gelée!

## define function to save a recipe Markdown and process complete batch

### get and save recipe function

In [43]:
class Recipe(BaseModel):
    recipe: str


def get_and_save_recipe(recipe_str, filename):
  completion = client.beta.chat.completions.parse(
      model="gpt-4o-2024-08-06",
      messages=[
          {"role": "system", "content": "You are a Michelin starred chef. You are given a dish. Provide the recipe in Markdown format."},
          {"role": "user", "content": recipe_str},
      ],
      response_format=Recipe,
  )

  recipe = completion.choices[0].message.parsed.recipe
  with open(filename, "w") as markdown_file:
      markdown_file.write(recipe)


In [44]:
get_and_save_recipe(dish_str_list[1], menu_filenames[1])

### create export dir and add it to filenames

We want to store the recipes in a folder for an easier export

In [45]:
!mkdir dish_recipes

In [50]:
# add dir "dishes_recipes" to filenames
menu_filenames = ['./dish_recipes/' + filename for filename in menu_filenames]

### process whole batch

The sequential mode was too slow so we parallelized in batches of 10 recipes to get our md files.

In [51]:
# too slow, 1h30 planned for the sequential execution

# from tqdm.auto import tqdm

# for dish_str, filename in tqdm(zip(dish_str_list, menu_filenames), total=len(dish_str_list)):
#   get_and_save_recipe(dish_str, filename)

  0%|          | 0/289 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [52]:
import concurrent.futures
from tqdm.auto import tqdm

# Function to process a single task
def process_task(dish_str, filename):
    get_and_save_recipe(dish_str, filename)

# Using ThreadPoolExecutor to run tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Use tqdm to show progress
    futures = []
    for dish_str, filename in zip(dish_str_list, menu_filenames):
        futures.append(executor.submit(process_task, dish_str, filename))

    # Wrap the futures in tqdm to show the progress bar
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(dish_str_list)):
        future.result()  # This ensures that exceptions are raised if they occur


  0%|          | 0/289 [00:00<?, ?it/s]

In [53]:
import shutil

# Path to the folder you want to zip
folder_path = '/content/dish_recipes'

# Path to the output zip file
output_zip = '/content/dish_recipes.zip'

# Zipping the folder
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', folder_path)

# Download the zip file
from google.colab import files
files.download(output_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>